In [65]:
import itertools
from tree_from_assoc_rules import list_all_factors
from controller import generate_decision_tree
from tabulate import tabulate

In [66]:
# Define all factors - Ref(3)
all_factors = ["stay", "length_of_stay", "country", "review_group", "room_type"]

# Read file of possible values
possible_factor_values = list_all_factors()

In [67]:
# All possible permutations of all factors - Ref(4)
all_factors_permutations = []
for length in range(2, len(all_factors) + 1):
    all_factors_permutations += list(itertools.permutations(all_factors, length))

In [68]:
# Hardcoded some constants
mcda_type = "rating"
hotel_name = "Amari Phuket"
rule_year = None
rule_month = None

In [83]:
def collect_paths(node, route_nodes=[]):
    result = []
    has_child = len(node.children) > 0
    if has_child:
        for child_node in node.children:
            new_route_nodes = []
            new_route_nodes += route_nodes
            new_route_nodes.append(node)
            result += collect_paths(child_node, route_nodes=new_route_nodes)
    else:
        new_route_nodes = []
        new_route_nodes += route_nodes
        new_route_nodes.append(node)
        return [new_route_nodes]
    return result

# Generate decision tree for all permutations
count = 0
nodes = []
result_score_table = [["Outcome", "Factor_1", "Factor_2", "Factor_3", "Factor_4", "Factor_5", "Score"]]
for idx, permutation in enumerate(all_factors_permutations):
    # Calculate weights array
    factors_length = len(permutation)
    factor_weight = 1 / factors_length
    weights = [factor_weight for x in range(factors_length)]

    # Generate Decision Tree
    # print(permutation)
    root_node = generate_decision_tree(mcda_type, list(permutation), weights, hotel_name=hotel_name,
                                       rule_year=rule_year, rule_month=rule_month)
    nodes.append(root_node)

    # Prototype: calculate score on each tree
    # print(RenderTree(root_node, style=AsciiStyle()).by_attr())
    paths = collect_paths(root_node)
    for p_idx, path in enumerate(paths):
        count += 1
        result_name = list(map(lambda k : k.name, path[1:]))
        weight_sum = sum(n.weight for n in path[1:])
        score_sum = sum(n.score for n in path[1:]) if weight_sum == 1 else 0
        result_row = [None] * 7
        result_row[0] = count
        result_row[1:5] = result_name + [None] * (5 - len(result_name))
        result_row[6] = score_sum
        result_score_table.append(result_row)

In [84]:
import csv

print(tabulate(result_score_table, headers='firstrow', tablefmt='fancy_grid'))
with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(result_score_table)

╒════╤══════════════╤════════════════════╤═══════════════════╤═════════════════════╤════════════════════════════════════════════════╤════════════╤═════════╕
│    │ Outcome      │ Factor_1           │ Factor_2          │ Factor_3            │ Factor_4                                       │   Factor_5 │ Score   │
╞════╪══════════════╪════════════════════╪═══════════════════╪═════════════════════╪════════════════════════════════════════════════╪════════════╪═════════╡
│  1 │ stay:DEC-FEB │ length_of_stay:1-7 │ country:Singapore │ review_group:Family │ room_type:Deluxe Ocean View                    │   0.196639 │         │
├────┼──────────────┼────────────────────┼───────────────────┼─────────────────────┼────────────────────────────────────────────────┼────────────┼─────────┤
│  2 │ stay:DEC-FEB │ length_of_stay:1-7 │ country:Singapore │ review_group:Family │ room_type:Superior Double Room with Ocean View │   0.196639 │         │
├────┼──────────────┼────────────────────┼────────────────